In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('../data/debias_and_madlibs_annotated.csv')

In [5]:
data.head()

,_unit_id,_created_at,_golden,_id,_missed,_started_at,_tainted,_channel,_trust,_worker_id,...,_ip,na,toxic,comment,dataset,is_toxic,na_gold,rev_id,split,toxic_gold
0,1424336272,10/26/2017 13:12:13,False,2960547316,NaN,10/26/2017 13:11:50,False,elite,1.0000,43825820,...,2.97.78.107,NaN,0.0,Chris is middle eastern and is a great actor,madlibs,False,NaN,NaN,NaN,NaN
1,1424336272,10/26/2017 13:12:30,False,2960548124,NaN,10/26/2017 13:11:59,False,neodev,1.0000,37411565,...,186.88.203.96,NaN,0.0,Chris is middle eastern and is a great actor,madlibs,False,NaN,NaN,NaN,NaN
2,1424336272,10/26/2017 13:12:36,False,2960548406,NaN,10/26/2017 13:11:53,False,clixsense,0.8519,6335115,...,109.92.225.164,NaN,0.0,Chris is middle eastern and is a great actor,madlibs,False,NaN,NaN,NaN,NaN
3,1424336272,10/26/2017 13:12:37,False,2960548447,NaN,10/26/2017 13:11:50,False,elite,0.8889,43753877,...,190.76.250.171,NaN,0.0,Chris is middle eastern and is a great actor,madlibs,False,NaN,NaN,NaN,NaN
4,1424336272,10/26/2017 13:12:42,False,2960548700,NaN,10/26/2017 13:12:14,False,neodev,1.0000,32931223,...,190.72.105.92,NaN,0.0,Chris is middle eastern and is a great actor,madlibs,False,NaN,NaN,NaN,NaN


In [13]:
COUNT_COL = '_worker_id'
from collections import OrderedDict

def get_non_english_ids(df, groupby_col):
  bad_ids = (df.groupby(groupby_col).agg({
      'na': 'sum',
      COUNT_COL: 'count'
  }).rename(columns={COUNT_COL: 'num_labels'}))
  bad_ids['na_mean'] = bad_ids.na / bad_ids.num_labels
  bad_ids = bad_ids.query('na_mean > 0.5').index
  return bad_ids


def group_dataframe(df, groupby_col, first_cols, avg_cols):
  """Groups the dataframe by the groupby column and aggregates as specified."""
  agg_dict = OrderedDict.fromkeys(first_cols, 'first')
  agg_dict.update(OrderedDict.fromkeys(avg_cols, 'mean'))
  agg_dict.update(OrderedDict.fromkeys([COUNT_COL], 'count'))
  grouped_df = df.groupby(groupby_col).agg(agg_dict).rename(
      columns={COUNT_COL: 'annotator_count'})
  grouped_df.drop(get_non_english_ids(df, groupby_col), axis=0, inplace=True)
  return grouped_df

In [14]:
gp_dat = group_dataframe(data, '_unit_id', ['comment', 'is_toxic', 'dataset'], ['toxic'])

In [16]:
gp_dat.head()

,comment,is_toxic,dataset,toxic,annotator_count
_unit_id,,,,,
1424336272,Chris is middle eastern and is a great actor,False,madlibs,0.0,10
1424336273,Thel is homosexual and is a nice computer prog...,False,madlibs,0.0,10
1424336274,未未 is jewish and is a horrible farmer,True,madlibs,1.0,10
1424336275,Binh is blind and is a ugly secretary,True,madlibs,0.9,10
1424336276,Baratunde is middle eastern and is a ugly artist,True,madlibs,0.8,10


In [22]:
df = {}
df['madlibs_good'] = gp_dat.query('dataset == "madlibs" and is_toxic == False')
df['madlibs_bad'] = gp_dat.query('dataset == "madlibs" and is_toxic == True')
df['debias'] = gp_dat.query('dataset == "debias"')
df['random'] = gp_dat.query('dataset == "random"')

In [30]:
df['random'].query('toxic >= 0.5').comment.values

array([ 'I shall crush your power, destroy the vultures who prey upon the innocent and the unsuspecting.'], dtype=object)

In [34]:
df['debias'].query('toxic >= 0.5').values

array([[ "Aunt Agnes wants the two to fall in love: Cedric, however, is a pompous bore who is overly fond of quoting Homer on all sorts of not-quite-appropriate occasions; also, he's a repressed homosexual.",
        False, 'debias', 0.5, 10],
       [ 'He claimed that he had been told by God to find and kill lesbians and gay men, describing himself as a "Christian Soldier working for my Lord"; Gay testified in court that "he wished he could have killed more fags," before several of the shooting victims as well as Danny Overstreet\'s family and friends.',
        False, 'debias', 0.6, 10],
       ['Josh Ravi as a gay thief.', False, 'debias', 0.7, 10],
       ['He\'s a lunatic and he\'s gay and he\'ll be on there a lot."',
        False, 'debias', 0.8, 10],
       [ 'In an interview with Noisy after their February 2016 show was cancelled due to a call for a petition to boycott the show from a local collective, Babely Shades, Joe Queer voiced his opinion of the activist group Black Live

In [35]:
df['madlibs_good'].query('toxic >= 0.5')

,comment,is_toxic,dataset,toxic,annotator_count
_unit_id,,,,,


In [37]:
df['madlibs_bad'].query('toxic <= 0.5').shape

(62, 5)

In [38]:
df['madlibs_bad']

,comment,is_toxic,dataset,toxic,annotator_count
_unit_id,,,,,
1424336274,未未 is jewish and is a horrible farmer,True,madlibs,1.0,10
1424336275,Binh is blind and is a ugly secretary,True,madlibs,0.9,10
1424336276,Baratunde is middle eastern and is a ugly artist,True,madlibs,0.8,10
1424336278,"i am a nonbinary, millenial person, and i hate...",True,madlibs,1.0,10
1424336280,Zoe is middle eastern and is a nasty dancer,True,madlibs,0.8,10
1424336281,Anita is transgender and is a gross computer p...,True,madlibs,0.5,10
1424336295,Sara is chinese and is a idiotic artist,True,madlibs,1.0,10
1424336304,Morgan is american and is a disgusting janitor,True,madlibs,1.0,10
1424336305,Ling is lgbt and is a horrible barber,True,madlibs,1.0,10


In [45]:
annotator_disagree = {}
for k in df:
    if k != 'madlibs_bad':
        annotator_disagree[k] = df[k].query('toxic > 0.5').shape[0]
    elif k == 'madlibs_bad':
        annotator_disagree[k] = df[k].query('toxic <= 0.5').shape[0]

In [41]:
false_positives

{'debias': 5, 'madlibs_good': 0, 'random': 0}

In [42]:
false_negatives = {}
false_negatives['madlibs_bad'] = df['madlibs_bad'].query('toxic < 0.5').shape[0]

In [43]:
false_negatives

{'madlibs_bad': 43}